# Filtros sRNA ↔ mRNA target

Notebook reorganizado para facilitar o uso: primeiro você define as variáveis, depois executa as células na ordem.


## 1) Variáveis que você precisa definir

Edite apenas esta célula (caminhos e parâmetros).

In [ ]:
# =========================
# CONFIGURAÇÃO DO USUÁRIO
# =========================

# (A) Arquivo de predições (CSV)
PREDICOES_CSV = "Todas_as_predições_Orthology.csv"

# (B) Arquivos DESeq2 (TSV) — um por cepa/condição
DEG_FILES = [
    "biofilm_vs_plank_N315.deseq2.results.tsv",
    "biofilm_vs_plank_MW2.deseq2.results.tsv",
    "biofilm_vs_plank_MRSA252.deseq2.results.tsv",
    "biofilm_vs_plank_LAC.deseq2.results.tsv",
    "biofilm_vs_plank_BMB.deseq2.results.tsv",
    "biofilm_vs_plank_NRS385.deseq2.results.tsv",
]

# (C) Cutoff para considerar gene como DEG no DESeq2
PADJ_CUTOFF = 0.05

# (D) Consistência entre cepas:
# quantas cepas precisam ter sinal (>0 para up, <0 para down) para considerar consistente
MIN_STRAINS_CONSISTENT = 4

# (E) Filtros de energia/probabilidade (outliers) nas predições
FILTERS_OUTLIER = {
    "E_intaRNA_max": -2.44,              # manter E_intaRNA <= este valor
    "E_Rnaplex_max": -32.6,              # manter E_Rnaplex <= este valor
    "E_TargetRNA3_max": -5.13,           # manter E_TargetRNA3 <= este valor
    "Probability_TargetRNA3_min": 0.06,  # manter Probability_TargetRNA3 >= este valor
    "Probability_sRNARFTarget_min": 0.40 # manter Probability_sRNARFTarget >= este valor
}

# (F) Arquivos de módulo (STRING) e arestas (pesos)
MODULE_NODES_FILE = "Table S12 - nodes.txt"      # colunas esperadas: nodeName, nodeAttr
MODULE_EDGES_FILE = "Table S13- edges.txt"       # colunas esperadas: fromNode, toNode, weight

# (G) Saída
OUTPUT_CSV = "filtered_weight.csv"


## 2) Imports e funções auxiliares

In [ ]:
import os
import numpy as np
import pandas as pd

def _check_exists(path: str) -> None:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Arquivo não encontrado: {path}")

def load_predicoes(csv_path: str) -> pd.DataFrame:
    _check_exists(csv_path)
    df = pd.read_csv(csv_path)
    required_cols = [
        "sRNA", "Target",
        "E_intaRNA", "E_Rnaplex", "E_TargetRNA3",
        "Probability_TargetRNA3", "Probability_sRNARFTarget",
    ]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Colunas faltando em {csv_path}: {missing}")
    return df

def filter_outliers(pred_df: pd.DataFrame, f: dict) -> pd.DataFrame:
    mask = (
        (pred_df["E_intaRNA"] <= f["E_intaRNA_max"]) &
        (pred_df["E_Rnaplex"] <= f["E_Rnaplex_max"]) &
        (pred_df["E_TargetRNA3"] <= f["E_TargetRNA3_max"]) &
        (pred_df["Probability_TargetRNA3"] >= f["Probability_TargetRNA3_min"]) &
        (pred_df["Probability_sRNARFTarget"] >= f["Probability_sRNARFTarget_min"])
    )
    return pred_df.loc[mask].copy()

def load_deg_tables(file_paths, padj_cutoff: float) -> dict:
    deg_dfs = {}
    for fp in file_paths:
        _check_exists(fp)
        df = pd.read_csv(fp, sep="\t")
        if "gene_id" not in df.columns or "log2FoldChange" not in df.columns or "padj" not in df.columns:
            raise ValueError(
                f"Arquivo DEG {fp} precisa ter colunas: gene_id, log2FoldChange, padj"
            )
        deg_dfs[fp] = df.loc[df["padj"] <= padj_cutoff, ["gene_id", "log2FoldChange"]].copy()
    return deg_dfs

def merge_deg_all_strains(deg_dfs: dict) -> pd.DataFrame:
    merged = None
    for fp, df in deg_dfs.items():
        col_name = fp.replace("biofilm_vs_plank_", "").replace(".deseq2.results.tsv", "")
        tmp = df.rename(columns={"log2FoldChange": col_name})
        if merged is None:
            merged = tmp
        else:
            merged = pd.merge(merged, tmp, on="gene_id", how="outer")
    return merged

def compute_regulation_status(deg_all: pd.DataFrame, min_strains: int) -> pd.DataFrame:
    numeric_only = deg_all.select_dtypes(include=[np.number])

    def status_row(row):
        pos = (row > 0).sum()
        neg = (row < 0).sum()
        if pos >= min_strains:
            return "upregulated"
        if neg >= min_strains:
            return "downregulated"
        return np.nan

    out = deg_all.copy()
    out["regulation_status"] = numeric_only.apply(status_row, axis=1)
    return out.dropna(subset=["regulation_status"]).copy()

def annotate_deg(pred_df: pd.DataFrame, deg_status_df: pd.DataFrame) -> pd.DataFrame:
    mapping = deg_status_df.set_index("gene_id")["regulation_status"].to_dict()
    out = pred_df.copy()
    out["sRNA_DEG"] = out["sRNA"].map(mapping)
    out["Target_DEG"] = out["Target"].map(mapping)
    return out

def load_modules(nodes_fp: str) -> pd.DataFrame:
    _check_exists(nodes_fp)
    df = pd.read_csv(nodes_fp, sep="\t")
    if "nodeName" not in df.columns or "nodeAttr" not in df.columns:
        raise ValueError(f"{nodes_fp} precisa ter colunas: nodeName, nodeAttr")
    return df

def annotate_modules(pred_df: pd.DataFrame, modules_df: pd.DataFrame) -> pd.DataFrame:
    mapping = modules_df.set_index("nodeName")["nodeAttr"].to_dict()
    out = pred_df.copy()
    out["sRNA_Module"] = out["sRNA"].map(mapping)
    out["Target_Module"] = out["Target"].map(mapping)
    return out

def load_edges(edges_fp: str) -> pd.DataFrame:
    _check_exists(edges_fp)
    df = pd.read_csv(edges_fp, sep="\t")
    required = {"fromNode", "toNode", "weight"}
    if not required.issubset(df.columns):
        raise ValueError(f"{edges_fp} precisa ter colunas: {sorted(required)}")
    df = df.copy()
    df["pair"] = df.apply(lambda r: tuple(sorted([r["fromNode"], r["toNode"]])), axis=1)
    return df[["pair", "weight"]]

def pick_best_weight_per_target(filtered_df: pd.DataFrame, edges_df: pd.DataFrame) -> pd.DataFrame:
    tmp = filtered_df.copy()
    tmp["pair"] = tmp.apply(lambda r: tuple(sorted([r["sRNA"], r["Target"]])), axis=1)
    merged = tmp.merge(edges_df, on="pair", how="left")
    merged = merged.sort_values(by=["Target", "weight"], ascending=[True, False])
    merged = merged[~merged["weight"].isna()].drop_duplicates(subset="Target", keep="first").reset_index(drop=True)
    return merged.drop(columns=["pair"])


## 3) Rodar pipeline

In [ ]:
# 3.1) Carregar predições
predicoes_df = load_predicoes(PREDICOES_CSV)

unique_srna_count = (
    predicoes_df["sRNA"]
    .dropna()
    .loc[predicoes_df["sRNA"].str.startswith("srn_", na=False)]
    .nunique()
)
print(f"Number of unique sRNA (prefixo 'srn_'): {unique_srna_count}")
print(f"Total de linhas (predições): {len(predicoes_df):,}")
predicoes_df.head()


In [ ]:
# 3.2) Filtro de outliers (energias/probabilidades)
predicoes_filtro_outlier_df = filter_outliers(predicoes_df, FILTERS_OUTLIER)
print(f"Após filtro outlier: {len(predicoes_filtro_outlier_df):,} linhas")
predicoes_filtro_outlier_df.head()


In [ ]:
# 3.3) Carregar DEGs (DESeq2), fazer merge e calcular consistência
deg_dfs = load_deg_tables(DEG_FILES, PADJ_CUTOFF)
deg_all = merge_deg_all_strains(deg_dfs)

deg_consistente = compute_regulation_status(deg_all, MIN_STRAINS_CONSISTENT)
print(f"Genes com regulação consistente (>= {MIN_STRAINS_CONSISTENT} cepas): {len(deg_consistente):,}")
deg_consistente.head()


In [ ]:
# 3.4) Anotar predições com status de DEG (sRNA e Target)
predicoes_anot_deg = annotate_deg(predicoes_filtro_outlier_df, deg_consistente)

# manter apenas pares onde sRNA e Target aparecem como DEG consistente
predicoes_filtro_DEG_df = predicoes_anot_deg.dropna(subset=["sRNA_DEG", "Target_DEG"]).copy()
print(f"Após filtro DEG (sRNA e Target com status): {len(predicoes_filtro_DEG_df):,} linhas")
predicoes_filtro_DEG_df.head()


In [ ]:
# 3.5) Carregar módulos e filtrar por mesmo módulo
modules_df = load_modules(MODULE_NODES_FILE)
predicoes_anot_mod = annotate_modules(predicoes_filtro_DEG_df, modules_df)

filtered_df = predicoes_anot_mod.loc[
    predicoes_anot_mod["sRNA_Module"] == predicoes_anot_mod["Target_Module"]
].copy()

print(f"Após filtro mesmo módulo: {len(filtered_df):,} linhas")
filtered_df.head()


In [ ]:
# 3.6) Estatísticas rápidas
unique_sRNAs = filtered_df["sRNA"].nunique()
unique_genes = filtered_df["Target"].nunique()
mean_sRNAs_per_gene = filtered_df.groupby("Target")["sRNA"].nunique().mean()

print(f"Number of unique sRNAs: {unique_sRNAs}")
print(f"Number of unique genes (targets): {unique_genes}")
print(f"Mean number of sRNAs predicted for each gene: {mean_sRNAs_per_gene:.2f}")


In [ ]:
# 3.7) Trazer peso (edges) e selecionar melhor sRNA por target
edges_df = load_edges(MODULE_EDGES_FILE)
filter_df = pick_best_weight_per_target(filtered_df, edges_df)

print(f"Após selecionar melhor peso por Target: {len(filter_df):,} linhas")
filter_df.head()


In [ ]:
# 3.8) Salvar resultado
filter_df.to_csv(OUTPUT_CSV, index=False)
print(f"Salvo em: {OUTPUT_CSV}")
